In [32]:
!pip install nltk

You are using pip version 10.0.1, however version 21.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.



  Using cached https://files.pythonhosted.org/packages/92/75/ce35194d8e3022203cca0d2f896dbb88689f9b3fce8e9f9cff942913519d/nltk-3.5.zip
  Using cached https://files.pythonhosted.org/packages/34/5b/bd0f0fb5564183884d8e35b81d06d7ec06a20d1a0c8b4c407f1554691dce/joblib-1.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4f/3f/40c8db23e022ccc9eb9fc0f39202af49c8614b22990b2e7129c2543f2da5/regex-2020.11.13-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/80/02/8f8880a4fd6625461833abcf679d4c12a44c76f9925f92bf212bb6cefaad/tqdm-4.56.0-py2.py3-none-any.whl
  Running setup.py install for nltk: started
    Running setup.py install for nltk: finished with status 'done'


In [1]:
from elasticsearch import Elasticsearch 
import json
import nltk
from nltk.tokenize import word_tokenize
import itertools
from collections import Counter
import string

In [2]:
es=Elasticsearch([{'host':'127.0.0.1','port':9200}])

In [4]:
def search_by_screen_name(index, name, es):
    res= es.search(index=index,body=
    {
      "query": {
        "match": {
          "screen_name": name
        }
      }
    })
    return res

In [5]:
res = search_by_screen_name("users", "BarackObama", es)

In [6]:
def flat_list(l):
    return  [item for sublist in l for item in sublist]

def common_tokens(tokens, n=20):
    sentences = (list(itertools.chain(tokens)))
    flat_sentences = flat_list(sentences)
    counts = Counter(flat_sentences)
    
    return counts

In [7]:
list_tokens=[]
stop_words=nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

for text in (res["hits"]["hits"][0]["_source"]["full_text"]):
    tokens_clean = []
    for word in word_tokenize(text):
        if word.lower() not in stop_words and word.lower() not in punctuation and not word.isnumeric() and len(word)> 1:
            tokens_clean.append(word.lower())
    list_tokens.append(tokens_clean)

In [8]:
counts = common_tokens(list_tokens)

In [9]:
counts.most_common(30)

[('https', 5),
 ('day', 4),
 ('one', 3),
 ('aaron', 2),
 ('ever', 2),
 ('people', 2),
 ('met', 2),
 ('climate', 2),
 ('today', 2),
 ('like', 2),
 ("'re", 2),
 ('brave', 2),
 ('enough', 2),
 ('hank', 1),
 ('best', 1),
 ('baseball', 1),
 ('players', 1),
 ('seen', 1),
 ('strongest', 1),
 ('michelle', 1),
 ('send', 1),
 ('thoughts', 1),
 ('prayers', 1),
 ('family', 1),
 ('everyone', 1),
 ('inspired', 1),
 ('unassuming', 1),
 ('man', 1),
 ('towering', 1),
 ('example', 1)]

In [64]:
counts.most_common(30)

[('.', 15),
 ('and', 10),
 ('the', 9),
 (':', 6),
 (',', 6),
 ('you', 6),
 ('https', 5),
 ('a', 5),
 ('was', 4),
 ('we', 4),
 ('is', 4),
 ('for', 4),
 ('day', 4),
 ('only', 4),
 ('because', 4),
 ('one', 3),
 ('of', 3),
 ('our', 3),
 ('to', 3),
 ('this', 3),
 ('that', 3),
 ('it', 3),
 ('aaron', 2),
 ('’', 2),
 ('ve', 2),
 ('ever', 2),
 ('people', 2),
 ('i', 2),
 ('met', 2),
 ('by', 2)]